# Getting familiar with LIANA

`liana` provides different statistical methods to infer `ligand-receptor` interactions from single-cell transcriptomics data omics data using prior knowledge.
In this notebook we showcase how to use liana in its most basic form with toy data.



In [ ]:
import scanpy as sc
import liana as li

## load example dataset

In [ ]:
adata = sc.datasets.pbmc68k_reduced()

`liana` typically works with the log1p-trasformed counts matrix, in this object the normalized counts are stored in `raw`:

In [ ]:
adata.raw.X

In [ ]:
sc.pl.umap(adata, color='bulk_labels', title='', frameon=False)

## Methods

In [ ]:
li.mt.show_methods()

Each method infers relevant ligand-receptor interactions relying on different assumptions and each method returns different ligand-receptor scores, typically a pair per method. One score corresponding to
the `magnitude` (strength) of interaction and the other reflecting how `specificity` of a given interaction to a pair cell identities.

## Resources

In [ ]:
li.rs.show_resources()

By default, liana uses the consensus resource, which is composed by multiple expert-curated ligand-receptor resources, including CellPhoneDB, CellChat, ICELLNET, connectomeDB2020, and CellTalkDB.

In [ ]:
from liana.mt import cellchat
?cellchat.__call__

## combine the scores of multiple methods

In [ ]:
from liana.mt import rank_aggregate

In [ ]:
rank_aggregate.describe()

In [ ]:
# import all individual methods
from liana.method import singlecellsignalr, connectome, cellphonedb, natmi, logfc, cellchat, geometric_mean

## run cellphonedb

In [ ]:
cellphonedb(adata,
            groupby='bulk_labels', 
            # NOTE by default the resource uses HUMAN gene symbols
            resource_name='consensus',
            expr_prop=0.1,
            verbose=True, key_added='cpdb_res')

In [ ]:
# by default, liana's output is saved in place:
adata.uns['cpdb_res'].head()

### Visualization

In [ ]:
li.pl.dotplot(adata = adata, 
              colour='lr_means',
              size='cellphone_pvals',
              inverse_size=True, # we inverse sign since we want small p-values to have large sizes
              source_labels=['CD34+', 'CD56+ NK', 'CD14+ Monocyte'],
              target_labels=['CD34+', 'CD56+ NK'],
              figure_size=(8, 7),
              # finally, since cpdbv2 suggests using a filter to FPs
              # we filter the pvals column to <= 0.05
              filter_fun=lambda x: x['cellphone_pvals'] <= 0.05,
              uns_key='cpdb_res' # uns_key to use, default is 'liana_res' 
             )

In [ ]:
my_plot = li.pl.tileplot(adata = adata, 
                         # NOTE: fill & label need to exist for both
                         # ligand_ and receptor_ columns
                         fill='means',
                         label='props',
                         label_fun=lambda x: f'{x:.2f}',
                         top_n=10, 
                         orderby='cellphone_pvals',
                         orderby_ascending=True,
                         source_labels=['CD34+', 'CD56+ NK', 'CD14+ Monocyte'],
                         target_labels=['CD34+', 'CD56+ NK'],
                         uns_key='cpdb_res', # NOTE: default is 'liana_res'
                         source_title='Ligand',
                         target_title='Receptor',
                         figure_size=(8, 7)
                         )
my_plot

## Ranking and aggregating

In [ ]:
# Run rank_aggregate
li.mt.rank_aggregate(adata, 
                     groupby='bulk_labels',
                     resource_name='consensus',
                     expr_prop=0.1,
                     verbose=True)


In [ ]:
?li.mt.rank_aggregate

In [ ]:
adata.uns['liana_res'].head()

### Visualization

In [ ]:
li.pl.dotplot(adata = adata, 
              colour='magnitude_rank',
              size='specificity_rank',
              inverse_size=True,
              inverse_colour=True,
              source_labels=['CD34+', 'CD56+ NK', 'CD14+ Monocyte'],
              target_labels=['CD34+', 'CD56+ NK'],
              top_n=10, 
              orderby='magnitude_rank',
              orderby_ascending=True,
              figure_size=(9, 5)
             )

In [ ]:
my_plot = li.pl.dotplot(adata = adata, 
                        colour='magnitude_rank',
                        inverse_colour=True,
                        size='specificity_rank',
                        inverse_size=True,
                        source_labels=['CD34+', 'CD56+ NK', 'CD14+ Monocyte'],
                        target_labels=['CD34+', 'CD56+ NK'],
                        filter_fun=lambda x: x['specificity_rank'] <= 0.01,
                       )
my_plot

In [ ]:
#Save the plot to a file
my_plot.save('dotplot.pdf')

## Customizing LIANA's rank aggregate

In [ ]:
li.mt.show_methods()


In [ ]:
# connectome, cellphonedb, natmi, logfc, cellchat, geometric_mean
methods = [cellchat, logfc,connectome]
new_rank_aggregate = li.mt.AggregateClass(li.mt.aggregate_meta, methods=methods)


In [ ]:
new_rank_aggregate(adata,
                   groupby='bulk_labels',
                   expr_prop=0.1, 
                   verbose=True,
                   n_perms=500,
                   #n_perms=None, # you can also don't perform permutations to increase the speed, and exclude p-values and specificity_rank
                   use_raw=True,
                   )


In [ ]:
adata.uns['liana_res'].head() 

In [ ]:
li.pl.dotplot(adata = adata, 
              colour='magnitude_rank',
              size='specificity_rank',
              inverse_size=True,
              inverse_colour=True,
              source_labels=['CD34+', 'CD56+ NK', 'CD14+ Monocyte'],
              target_labels=['CD34+', 'CD56+ NK', 'CD14+ Monocyte'],
              top_n=10, 
              orderby='magnitude_rank',
              orderby_ascending=True,
              figure_size=(9, 5)
             )

### Customizing LIANA's Plots

In [ ]:
import plotnine as p9

In [ ]:
(my_plot +
 # change theme
 p9.theme_dark() +
 # modify theme
 p9.theme(
     # adjust facet size
     strip_text=p9.element_text(size=11),
     figure_size=(7, 4)
 )
)

In [ ]:
# Circle Plot
li.pl.circle_plot(adata,
                  groupby='bulk_labels',
                  score_key='magnitude_rank',
                  inverse_score=True,
                  source_labels='CD34+',
                 # filter_fun=lambda x: x['specificity_rank'] <= 0.05,
                  pivot_mode='counts', # NOTE: this will simply count the interactions, 'mean' is also available
                  figure_size=(10, 10),
                  )


# Practice

Use the use an ischemic 10X Visium spatial slide from [Kuppe et al., 2022] `kuppe_heart19.h5ad` dataset to practice:
- calculating ligand-receptor interactions per cell type using rank_aggregate method;
- visualize the top 10 ligand-receptor interactions of each cell type;

Tips: 
- The cell-type compositions per spot are stored in obsm['compositions']. You can assign each spot to the cell type with the highest value in 'compositions' by setting adata.obs['cell_type'].
- Don't forget to normalize the data before running LIANA.